In [38]:
import mne
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import re
import json
from queue import Queue

In [43]:


BUFFER_SIZE = 6000 # 30s of frames at 200 Hz (Assuming 1 frame at 200 Hz) = 30 s x 200 Hz/s = 6000 frames 
data_queue = Queue(maxsize = BUFFER_SIZE) # Initialize Queue

def collect_data(packet):
    """
    This method collects 30 seconds worth of packets from the web socket for processing.

    Parameters:
    - Packet: Frame sent by the NeuraSense app.

    Returns:
    """

    # Dump out queue for processing if full
    if data_queue.full():

        # Size should be BUFFER_SIZE
        size = data_queue.qsize() 

        # Initialize array to collect data chunk
        data_chunk = [] 

         # Dequeue frames to gather one data chunk for processing
        for i in range(size):
            data_chunk.append(data_queue.get_nowait()) 

        # Clear queue to collect next 30s of data
        data_queue.queue.clear() 

        # Send data chunk for processing
        return data_chunk

    else:
        # Add packets to queue until it fills up
        data_queue.put(packet)
        return None # Return None so the application can send more data


def processing_data (data):
    
    raw_data = f"""{data}""" #This needs to be data from the queue

    # Extract individual JSON objects and create data array
    json_objects = re.findall(r'\{.*?\}', raw_data, re.DOTALL)

    # Make it a valid JSON
    json_objects = [obj.replace("'", '"').replace("type:", '"type":').replace("data:", '"data":') for obj in json_objects]

    # Parse into a list of dictionaries
    parsed_data = [json.loads(obj) for obj in json_objects]

    num_channels = len(parsed_data[0]["data"])  # Assuming all JSONs have the same number of channels
    concatenated_data = [[] for _ in range(num_channels)]  # List of lists for each channel

    # Append corresponding channel samples from each JSON object
    for entry in parsed_data:
        for ch_idx in range(num_channels):
            concatenated_data[ch_idx].extend(entry["data"][ch_idx])  # Concatenate channel-wise data

    # Convert to a NumPy array
    data_array = np.array(concatenated_data)  # Shape: (n_channels, n_times)

    # Create MNE raw object
    sfreq = 200  # Set the sampling frequency in Hz (Modify as needed)
    ch_names = [f"channel_{i+1}" for i in range(num_channels)]
    ch_types = ["eeg"] * num_channels  # Set channel types

    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    raw_mne_data = mne.io.RawArray(data_array, info)
    #raw_mne_data.set_eeg_reference(ref_channels=['Cz']) # Need to determine how reference channel will be communicated and update

    # Computing psds (mean band power)
    #channels = raw_mne_data.info['ch_names'] ##May need to update when using reference
    #[channel_1, channel_2, channel_3, channel_4]
    
    # Dummy Data    
    # Arranging data into format required for application
    DAR = np.float64(93.26683132320454) #band_ratios['DAR']
    DBR = np.float64(356.5805171124732) #band_ratios['DBR']
    RBP_Alpha = np.float64(0.015500846730665504) #relative_band_power['Alpha']
    RBP_Beta = np.float64(0.004266456282917205) #relative_band_power['Beta']
    RD_Alpha = np.float64(0.046607329662870435) #relative_diff['Alpha']
    RD_Beta = np.float64(0.006460576685520055) #relative_diff['Beta']
    HI_Alpha = np.float64(0.910936359144583) #hemispheric_index['Alpha']
    HI_Beta =  np.float64(0.9871617888764286) #hemispheric_index['Beta']
    result = 'Stroke Assessment: High'
    # Conduct stroke assessment with flag-based algorithm
    
    
    return DAR, DBR, RBP_Alpha, RBP_Beta, RD_Alpha, RD_Beta, HI_Alpha, HI_Beta, result

In [ ]:
def main_loop():
    """
    Simulates the web app sending packets continuously.
    When a full chunk is collected, it processes the data.
    """
    # This loop could be event-driven or threaded in a real app.
    while True:
        # Here, simulate receiving a packet from the web socket.
        packet = {
            "type": "timeSeriesFilt",
            "data": [
                [0.5028886,   1.2675915,  2.4512942, -0.57264835, -2.363007, -0.44036055, 1.6118289,   1.8887416],
                [0.65561265,  0.70927393,  1.8596437, -0.79762936, -0.7252633,  0.50478804, 1.7553623,   2.7865443],
                [0.052701943, -1.9235382, -3.4246593, -2.3290026,  0.27224013,  1.6203736, 1.9227136,   3.8821752],
                [3.1687765,   4.0005245,  4.02343,   -0.8285941, -1.5729312, -0.33332944, 1.0582756,   0.091916144]
            ]
        }
        
        # The collect_data function returns a data chunk when the queue is full,
        # or None if it's still collecting.
        data_chunk = collect_data(packet)
        if data_chunk is not None:
            # Process the collected data.
            results = processing_data(data_chunk)
            print("Processing results:", results)
            # Here you might also send the results back to the web app or elsewhere.


# To test the workflow, uncomment the following line:
main_loop()


Creating RawArray with float64 data, n_channels=4, n_times=48000
    Range : 0 ... 47999 =      0.000 ...   239.995 secs
Ready.
Processing results: (np.float64(93.26683132320454), np.float64(356.5805171124732), np.float64(0.015500846730665504), np.float64(0.004266456282917205), np.float64(0.046607329662870435), np.float64(0.006460576685520055), np.float64(0.910936359144583), np.float64(0.9871617888764286), 'Stroke Assessment: High')
Creating RawArray with float64 data, n_channels=4, n_times=48000
    Range : 0 ... 47999 =      0.000 ...   239.995 secs
Ready.
Processing results: (np.float64(93.26683132320454), np.float64(356.5805171124732), np.float64(0.015500846730665504), np.float64(0.004266456282917205), np.float64(0.046607329662870435), np.float64(0.006460576685520055), np.float64(0.910936359144583), np.float64(0.9871617888764286), 'Stroke Assessment: High')
Creating RawArray with float64 data, n_channels=4, n_times=48000
    Range : 0 ... 47999 =      0.000 ...   239.995 secs
Ready

KeyboardInterrupt: 